<a href="https://colab.research.google.com/github/Jhc-china/Learning-Notebook/blob/master/Tensorflow2_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow 2.0

该文件为Tensorflow 2.0 的学习笔记

## Keras

在Tensorflow中Keras是用于构建和训练深度学习模型的高级API

其三个优点为：
1. 方便用户使用
2. 模块化和可组合
3. 易于扩展

下面将介绍Tensorflow中常见的Keras模块的操作

### Keras模块的导入

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

1.14.0
2.2.4-tf


### 构建简单模型

#### 模型的堆叠

常见的模型堆叠是层的堆叠，采用tf.keras.Sequential模型

In [0]:
model = tf.keras.Sequential()
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 上述步骤与下面步骤等价

model = tf.keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])

#### 模型每一层的配置

tf.keras.layers中每一层的配置常见的有：
1. activation：设置层的激活函数。此参数由内置函数的名称指定，或指定为可调用对象。默认情况下，系统不会应用任何激活函数
2. kernel_initializer和bias_initializer：创建层权重（核和偏差）的初始化方案。此参数是一个名称或可调用对象，默认为 "Glorot uniform" 初始化器
3. kernel_regularizer和bias_regularizer：应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数

In [11]:
# 例如可以像下面进行配置
layers.Dense(32, activation='sigmoid')
layers.Dense(32, activation=tf.sigmoid)
layers.Dense(32, kernel_initializer='orthogonal')
layers.Dense(32, kernel_initializer=tf.keras.initializers.glorot_normal)
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01))
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l1(0.01))

### 训练和评估模型

模型构造好后，我们可以对模型进行配置，主要使用model的compile方法

#### 模型配置

在model具体的build后者fit输入相关数据之前，我们可以查看模型的具体每层的形状，调用model.summary()方法即可

不过要注意的是，必须先指定好输入层的input_shape才行

In [0]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

#### 可输入Numpy数据

keras构造的model模型，可以直接通过numpy的数据类型进行训练

np.random.random将生成在[0,1)的半开半闭取件内的浮点数，与random.rand不同的是，后者生成[0,1]区间的小数

输入参数为生成np array的size

下面是一个简单的numpy数据输入训练过程，用于训练的x数据为1000组每组长度为72的随机数据，真值y为对应1000组长度为10的随机数据

In [16]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/10
1000/1000 [==============================] - 0s 27us/sample - loss: 195.9979 - categorical_accuracy: 0.0820 - val_loss: 210.3744 - val_categorical_accuracy: 0.1350
Epoch 2/10
1000/1000 [==============================] - 0s 25us/sample - loss: 220.4191 - categorical_accuracy: 0.0810 - val_loss: 235.2692 - val_categorical_accuracy: 0.0650
Epoch 3/10
1000/1000 [==============================] - 0s 22us/sample - loss: 245.5966 - categorical_accuracy: 0.0930 - val_loss: 261.5547 - val_categorical_accuracy: 0.1000
Epoch 4/10
1000/1000 [==============================] - 0s 24us/sample - loss: 272.0299 - categorical_accuracy: 0.0910 - val_loss: 288.3384 - val_categorical_accuracy: 0.1050
Epoch 5/10
1000/1000 [==============================] - 0s 24us/sample - loss: 300.0116 - categorical_accuracy: 0.0770 - val_loss: 317.2724 - val_categorical_accuracy: 0.1050
Epoch 6/10
1000/1000 [==============================] - 0s 24us/sample - loss:

#### 可利用tf.data输入数据

可利用tf.data.Dataset生成用于训练的对象，同时可以进行一些训练设置的操作

In [20]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.shuffle(1000)
dataset = dataset.batch(32)
dataset = dataset.repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 6ms/step - loss: 226.5335 - categorical_accuracy: 0.0938 - val_loss: 173.5174 - val_categorical_accuracy: 0.0625
Epoch 2/10
30/30 [==============================] - 0s 2ms/step - loss: 122.1430 - categorical_accuracy: 0.1036 - val_loss: 83.2841 - val_categorical_accuracy: 0.1562
Epoch 3/10
30/30 [==============================] - 0s 2ms/step - loss: 69.4316 - categorical_accuracy: 0.1068 - val_loss: 109.2683 - val_categorical_accuracy: 0.0833
Epoch 4/10
30/30 [==============================] - 0s 2ms/step - loss: 105.1302 - categorical_accuracy: 0.1186 - val_loss: 61.0480 - val_categorical_accuracy: 0.0833
Epoch 5/10
30/30 [==============================] - 0s 2ms/step - loss: 82.6610 - categorical_accuracy: 0.0951 - val_loss: 59.2898 - val_categorical_accuracy: 0.0833
Epoch 6/10
30/30 [==============================] - 0s 2ms/step - loss: 133.1314 - categorical_accuracy: 0.0972 - val_loss: 161.4064 - val_categorical_accuracy: 0.15

#### 模型的评估和预测

我们可以通过测试集数据对训练的模型效果进行评估，主要采用model.evaluate方法

若要使用模型对数据进行结果预测，则主要采用model.predict方法

In [21]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)  # 采用这种操作将对1000组数据逐个进行测试得到结果
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)  # 采用这种操作会执行30次测试，每次测试一个batch(batch_size上面设置为32)

30/30 [==============================] - 0s 3ms/step - loss: 137.6307 - categorical_accuracy: 0.0948


[137.63073806762696, 0.094791666]

In [24]:
result = model.predict(test_x, batch_size=32)
print(result)

[[3.78529694e-06 1.02617650e-10 2.03043868e-31 ... 9.99988794e-01
  1.17900803e-22 5.88397207e-08]
 [5.28550800e-07 2.69229561e-12 1.73206051e-36 ... 9.99998093e-01
  6.67561030e-26 6.05805628e-09]
 [8.38047299e-07 2.51477242e-11 6.41219771e-34 ... 9.99996185e-01
  2.66655812e-24 2.43373464e-08]
 ...
 [2.50193489e-06 4.55364219e-11 2.14007195e-32 ... 9.99991298e-01
  3.91845955e-23 4.61839420e-08]
 [2.62199796e-06 1.09235884e-10 1.83820835e-30 ... 9.99988437e-01
  4.69963846e-22 7.09742238e-08]
 [9.34907575e-07 1.85427333e-11 3.66124246e-33 ... 9.99994278e-01
  8.54075247e-24 3.30062733e-08]]
(1000, 10)


### 高级模型构建

#### 函数式API

前面采用的tf.keras.Sequential是模型的简单堆叠，无法用来表示任意的模型，而使用Keras函数式API可以构造复杂的模型拓扑，比如：

多输入模型

多输出模型

具有共享层的模型(同一层被调用多次)

具有非序列数据流的模型(例如：残差连接)

#### 使用函数式API构建的模型有以下特点：

层实例可以调用并且返回张量，输入张量和输出张量用于定义 tf.keras.Model 实例，此模型的训练方式和 Sequential 模型一样

In [26]:
input_x = tf.keras.Input(shape=(72,)) # 输入层
hidden1 = layers.Dense(32, activation='relu')(input_x)  # 连接在第一个输入层之后的第一个隐藏层
hidden2 = layers.Dense(64, activation='relu')(hidden1)  # 连接在第一个隐藏层之后的第二个隐藏层
pred = layers.Dense(10, activation='softmax')(hidden2)  # 连接在第二个隐藏层之后的输出预测层

model = tf.keras.Model(inputs=input_x, outputs=pred)  # 可以定义tf.keras.Model实例，类似于之前用Sequential定义的模型
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 129us/sample - loss: 13.0147 - acc: 0.0980
Epoch 2/5
1000/1000 [==============================] - 0s 50us/sample - loss: 20.8857 - acc: 0.0950
Epoch 3/5
1000/1000 [==============================] - 0s 51us/sample - loss: 42.6007 - acc: 0.1050
Epoch 4/5
1000/1000 [==============================] - 0s 48us/sample - loss: 83.4281 - acc: 0.0980
Epoch 5/5
1000/1000 [==============================] - 0s 50us/sample - loss: 143.6454 - acc: 0.0950


#### 模型子类化

可以通过构建tf.keras.Model的子类，自己定义前向传播过程和模型，在__init__方法中可以设置类实例的属性，在call方法中可以定义前向传播的过程

涉及到类的方法重写

In [27]:
class MyModel(tf.keras.Model):
  # 定义自己的模型
  def __init__(self, num_classes=10):
    super().__init__(name='my_model')
    self.num_classes = num_classes
    self.layer1 = layers.Dense(32, activation='relu')
    self.layer2 = layers.Dense(num_classes, activation='softmax')
  # 定义前向传播过程
  def call(self, inputs):
    h1 = self.layer1(inputs)
    out = self.layer2(h1)
    return out
  # 获取输出的shape
  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.num_classes
    return tf.tf.TensorShape(shape)
  
model = MyModel(num_classes=10)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 156us/sample - loss: 16.4664 - acc: 0.1020
Epoch 2/5
1000/1000 [==============================] - 0s 77us/sample - loss: 23.9011 - acc: 0.1020
Epoch 3/5
1000/1000 [==============================] - 0s 77us/sample - loss: 29.0692 - acc: 0.1060
Epoch 4/5
1000/1000 [==============================] - 0s 83us/sample - loss: 32.8826 - acc: 0.0980
Epoch 5/5
1000/1000 [==============================] - 0s 76us/sample - loss: 35.9737 - acc: 0.1060


#### 自定义层

类似于上面对于Model的子类化，可以通过对于layers.Layers的子类化来自定义自己的层，一般包括：
1. build：创建层的权重。使用 add_weight 方法添加权重
2. call：定义前向传播
3. compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。 或者，可以通过实现 get_config 方法和 from_config 类方法序列化层

涉及到类的方法的重写

In [32]:
class MyLayer(layers.Layer):
  
  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super().__init__(**kwargs)
  # 创造层的权重
  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    self.kernel = self.add_weight(name='kernel1', shape=shape,
                                 initializer='uniform', trainable=True)
    super().build(input_shape)
  # 前向传播
  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)
  # 输出形状计算
  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)
  # 获取层的config
  def get_config(self):
    base_config = super().get_config()
    base_config['output_dim'] = self.output_dim
    return base_config
  # 从config实例化层
  @classmethod
  def from_config(cls, config):
    return cls(**config)

model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

W0707 14:52:14.135005 140231248828288 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/5
1000/1000 [==============================] - 0s 119us/sample - loss: 11.5639 - acc: 0.1130
Epoch 2/5
1000/1000 [==============================] - 0s 69us/sample - loss: 11.5628 - acc: 0.1080
Epoch 3/5
1000/1000 [==============================] - 0s 73us/sample - loss: 11.5630 - acc: 0.1150
Epoch 4/5
1000/1000 [==============================] - 0s 72us/sample - loss: 11.5625 - acc: 0.1070
Epoch 5/5
1000/1000 [==============================] - 0s 70us/sample - loss: 11.5628 - acc: 0.1090


#### 回调

在模型训练的过程中(即model.fit过程中)，我们可以增加回调函数，来增加一些方法(可以用已有的，可以自己写)，例如已有的EarlyStopping方法用于防止过拟合，Tensorboard方法用于增加Tensorboard的log信息

In [33]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

model.fit(train_x, train_y, batch_size=32, epochs=5,
          callbacks=callbacks, validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 0s 140us/sample - loss: 11.5620 - acc: 0.1110 - val_loss: 11.6872 - val_acc: 0.1150
Epoch 2/5
1000/1000 [==============================] - 0s 47us/sample - loss: 11.5617 - acc: 0.1110 - val_loss: 11.6882 - val_acc: 0.1200
Epoch 3/5
1000/1000 [==============================] - 0s 47us/sample - loss: 11.5626 - acc: 0.1120 - val_loss: 11.6865 - val_acc: 0.1300
Epoch 4/5
1000/1000 [==============================] - 0s 48us/sample - loss: 11.5619 - acc: 0.1100 - val_loss: 11.6855 - val_acc: 0.1100
Epoch 5/5
1000/1000 [==============================] - 0s 46us/sample - loss: 11.5612 - acc: 0.1070 - val_loss: 11.6856 - val_acc: 0.1250


### 模型的保存和恢复